<h1> TAXI DATA COLUMBIA </h1>
<h2> For this part, we look at taxi data going from columbia area to other places in 2015 </h2>

In [1]:
%matplotlib inline
import os
import seaborn as sns
import numpy as np
import pandas as pd
from pandas import DataFrame
from geopandas.tools import sjoin
import matplotlib.pylab as plt
import pysal as ps
import geopandas as gpd
from pandas.io.gbq import read_gbq
import datetime as dt
from mpl_toolkits.basemap import Basemap
import matplotlib.cm
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

In [6]:
project = "spheric-crow-161317"
cool_query = "select * from `bigquery-public-data.new_york.tlc_yellow_trips_2015` where ( \
(pickup_latitude between 40.802708 and 40.812708) and \
(pickup_longitude between -73.967248 and -73.957248) and \
(dropoff_latitude between 40.700473 and 40.764400) and \
(dropoff_longitude between -74.024549 and -73.953507))"

taxi_going= read_gbq(query=cool_query, project_id=project, dialect='standard')

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
  Got page: 1; 16% done. Elapsed 8.23 s.
  Got page: 2; 32% done. Elapsed 14.68 s.
  Got page: 3; 48% done. Elapsed 23.28 s.
  Got page: 4; 64% done. Elapsed 40.59 s.
  Got page: 5; 81% done. Elapsed 49.2 s.
  Got page: 6; 97% done. Elapsed 60.02 s.
  Got page: 7; 100% done. Elapsed 66.11 s.
Got 114261 rows.

Total time taken 72.84 s.
Finished at 2017-04-25 16:06:07.


In [9]:
taxi_going['pickup_weekday'] = taxi_going.pickup_datetime.dt.weekday

In [14]:
taxi_going['pickup_hour'] = taxi_going.pickup_datetime.dt.hour


In [15]:
taxi_going.to_csv('taxi_going_columbus.csv')

In [2]:
#taxi_going_2015 is lower manhattan
#start from here
data = pd.read_csv('taxi_going_columbus.csv')

In [4]:
import gmplot
drop_lon = list(data['dropoff_longitude'])
drop_lat = list(data['dropoff_latitude'])
gmap = gmplot.GoogleMapPlotter.from_geocode("Central Park",12)
gmap.heatmap(drop_lat, drop_lon)
gmap.draw('columbus.html')


<h2> Draw Heatmaps! </h2>
<h3> By Area</h3>
<li> Whole Manhattan </li>
Unfortunately, there are so many data, so the pattern is not obvious. We see data aggregated on the broadway
<li> Lower Manhattan (From Columbus circle and below) </li>
<h3> By Time </h3>
Ok, now we find lower Mahattan is worth looking at. We are interested in whether there is a difference respect to picking up time
<h4>Time in a day </h4>
<li>Morning</li>
4am-12pm
<li>Afternoon</li>
12pm-6pm
<li>Early Night</li>
6pm-10pm
<li>Late Night</li>
10pm-4am
<h4> Time in a week </h4>
<li>Weekday</li>
Monday to Thursday
<li>Weekend</li>
Friday to Sunday

In [4]:
weekday = data[data['pickup_weekday']<=3]
weekend = data[data['pickup_weekday']> 3]

In [25]:
boolv = list()
for hour in data['pickup_hour']:
    if hour > 4 and hour<=12:
        result = 0
        boolv.append(result)
    elif hour > 12 and hour <=18:
        result = 1
        boolv.append(result)
    elif hour>18 and hour <=22:
        result = 2
        boolv.append(result)
    else:
        boolv.append(3)
len(boolv)           

114261

In [26]:
data['time_category'] = boolv

In [27]:
morning = data[data['time_category']==0]
afternoon = data[data['time_category']==1]
early_night = data[data['time_category']==2]
late_night = data[data['time_category'] == 3]

In [3]:
def draw_heatmap(df,map_name):
    import gmplot
    drop_lon = list(df['dropoff_longitude'])
    drop_lat = list(df['dropoff_latitude'])
    gmap = gmplot.GoogleMapPlotter.from_geocode("Central Park",12)
    gmap.heatmap(drop_lat, drop_lon)
    gmap.draw(map_name)


In [29]:
draw_heatmap(morning,'columbus_morning.html')
draw_heatmap(afternoon,'columbus_afternoon.html')
draw_heatmap(early_night,'columbus_early_night.html')
draw_heatmap(late_night,'columbus_late_night.html')

In [8]:
draw_heatmap(weekday,'columbus_weekday.html')
draw_heatmap(weekend,'columbus_weekend.html')

In [2]:
df = pd.read_csv('taxi_going_man.csv')

In [4]:
draw_heatmap(df,'manhattan.html')